In [ ]:
# import modules and define functions
from pyscf import gto, scf
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import os
os.environ['OMP_NUM_THREADS'] = "1" 

def make_rdm1(mo_coeff, mo_occ):
    mocc = mo_coeff[:,mo_occ>0]
    return np.dot(mocc*mo_occ[mo_occ>0], mocc.conj().T)

In [ ]:
# define variables (bond length)
dmin, dmax = 0.5, 30.0
ds = np.linspace(dmin, dmax, 51)
ess = []
c1ss = []
c2ss = []
css = []
dss = np.array([ds.tolist()] * 101).T
for i, d in enumerate(ds):
    es = []
    c2s = []
    cs = []
    # build H2 at a given dH-H d with minimal basis
    atom = 'H 0 0 0; H 0 0 %s' %(d)
    basis = 'sto-3g'
    charge = 0
    spin = 0
    H2 = gto.Mole()
    H2.build(atom=atom,basis=basis,charge=charge,spin=spin)
    # initiate RHF
    mf = scf.UHF(H2)
    mf.max_cycle = 0
    # get overlap matrix at this bond length
    s = mf.get_ovlp(H2)
    # create a list of 51 c1 coefficients equally spaced between 0 and 1
    c1s = np.linspace(0,1,51)
    # loop through each c1 coefficient
    for c1 in c1s:
        # calculate c2 by c1**2 + c2**2 + c1*c2*S12 = 1
        c2 = (1+(c1**2)*(s[0][1]**2-1)) ** 0.5 - c1*s[0][1]
        # store the calculated c2
        c2s.append(c2)
        # calculate the density matrix using c1 and c2
        mo_coeff = np.array([[c1, c1], [c2, -c2]])
        mo_occ = np.array([2, 0])
        dm1 = make_rdm1(mo_coeff, mo_occ)
        # calculate E
        conv, e, mo_e, mo, mo_occ = scf.hf.kernel(mf, dm0=dm1)
        # store the calculated E
        es.append(e)
        # calculate the adjusted coefficient, c = c1**2+c1*c2*S12
        c = c1**2+c1*c2*s[0][1]
        # store the calculated c
        cs.append(c)
    css.append(cs)
    ess.append(es)
    c2ss.append(c2s)
    c1ss.append(c1s)
        

In [ ]:
# Plot an interactive 3D surface of energy at various adjusted coefficients given a H-H bond length
xlabel = 'c1**2+c1*c2*S12'
ylabel = 'Distance / A'
zlabel = 'Energy / Eh'
fig = make_subplots(rows=1, cols=1,
                    specs=[[{'is_3d': True}]],
                    subplot_titles=['Energy change w.r.t to mo coeff change at various dH-H'],
                    )

fig.add_trace(go.Surface(x=css, y=dss, z=ess), 1, 1) #, colorbar_x=-0.07), 1, 1)
fig.update_layout(
    scene = dict(
        xaxis_title = xlabel,
        yaxis_title = ylabel,
        zaxis_title = zlabel,
        xaxis = dict(
            nticks=5, 
            range=[0,1.0],
            ),
        yaxis = dict(
            tickvals=np.linspace(0,dmax,4),
            range=[0.3,dmax]
            ),
        zaxis = dict(
            tickvals=np.linspace(-1.2,0,4),
            range=[-1.2,0]
            ),
        aspectratio=dict(x=1, y=1, z=0.95)
    )
)
fig.show()